In [ ]:
import sqlite3
import requests
import pandas as pd
from pprint import pprint
from OTXv2 import OTXv2, IndicatorTypes

In [ ]:
# Connect to the database
connection = sqlite3.connect('2023-10-25_cti_data_majd.db')
cursor = connection.cursor()

In [ ]:
def get_table_columns(table_name, cursor):
    # Execute a query to get the column names for a specific table
    cursor.execute(f"PRAGMA table_info({table_name});")

    # Fetch the results
    columns = cursor.fetchall()

    # Return a list of column names
    return [column[1] for column in columns]

In [ ]:
# Pulses Table
query = 'SELECT * FROM pulses order by created'

cursor.execute(query)

# Fetch the results
pulse_results = cursor.fetchall()

# Get the columns
pulse_columns = get_table_columns('pulses', cursor)

# Dataframe
pulses = pd.DataFrame(pulse_results, columns=pulse_columns)
pulses.head()

In [ ]:
# indicators Table
query = 'SELECT * FROM indicators order by created'

cursor.execute(query)

# Fetch the results
indicators_results = cursor.fetchall()

# Get the columns
indicators_columns = get_table_columns('indicators', cursor)

# Dataframe
indicators = pd.DataFrame(indicators_results, columns=indicators_columns)
indicators.head()

In [ ]:
merged = pd.merge(indicators, ip_location, left_on='indicator', right_on='ip', how='outer').drop('ip', axis=1)

In [ ]:
# indicators Table
query = 'SELECT * FROM ip_location'

cursor.execute(query)

# Fetch the results
ip_results = cursor.fetchall()

# Get the columns
ip_columns = get_table_columns('ip_location', cursor)

# Dataframe
ip_location = pd.DataFrame(ip_results, columns=ip_columns)
ip_location

In [ ]:
def process_data(data, columns):
    # Ensure that the column names are valid keys
    valid_columns = [name.replace(" ", "_").lower() for name in columns]

    # Zip the valid column names with the data
    formated = [dict(zip(valid_columns, row)) for row in data]

    return formated

In [ ]:
otx_api_key = "b33bf61f62c7e7da86bb84481591a0844630323831ec210fd8c2e2efbfdc131a"
otx = OTXv2(otx_api_key)

In [ ]:
# Get all pulses
pulses = otx.getall()

In [ ]:
headers = {
    'Content-Type': 'application/json',
    'X-OTX-API-KEY': otx_api_key,
}
pulses_data = process_data(pulse_results, pulse_columns)

In [ ]:
# Delete each pulse
for i, pulse in enumerate(pulses):
    pulse_id = pulse['id']
    response = requests.get(f'https://otx.alienvault.com/api/v1/pulses/{pulse_id}/delete', headers=headers)
    if i % 100 == 0:
        print(i)

In [ ]:
try:
    for i, pulse_data in enumerate(pulses_data):
        pulse_name = pulse_data['name'] if len(pulse_data['name'])>= 5 else 'Undefined Name'

        pulse_indicators = merged[merged['pulse_id'] == pulse_data['id']][['indicator', 'type', 'created', 'content', 'title', 'description', 'expiration', 'is_active']].to_dict(orient='records')
        
        if not pulse_indicators:
            print(f"No indicators found for pulse: {pulse_name}")
            continue
        # pprint(pulse_data['id'])


        otx.create_pulse(
            name=pulse_name,
            public=True,
            indicators=pulse_indicators,
            tags=[],
            references=[],
        )
        if i % 100 == 0:
            print(i)


except Exception:
    pass

In [ ]:
x = requests.get("https://otx.alienvault.com/api/v1/pulses/54e7650013432a12badec7e7", {
    'X-OTX-API-KEY': otx_api_key,
})
pprint(x.json())

In [ ]:
# Close the connection
connection.close()